In [ ]:
from astropy.table import Table, Column
import numpy as np
import os
import sys
from astropy.io import fits
from utilities import get_immediate_subdirectories

sys.path.append(f'{os.environ["HOME"]}/Projects/planckClusters/catalogs')
from load_catalogs import load_PSZcatalog
from utilities import redshifts_from_papers

In [ ]:
results = Table.read('./catalogs/results_table_inspected_0.5_2.0.fits')

# filter out duplicates
results = results[~results['DUPLICATE']]

results_fields = [f.replace(' ', '_') for f in results['NAME']]

In [ ]:
fields = get_immediate_subdirectories('../data/')

In [ ]:
mask = [True for _ in fields]

for i, field in enumerate(fields):
    field = field.split('/')[-1]
    
    # there are two non-PSZ directories
    if field in results_fields or 'PSZ' not in field:
        mask[i] = False
    

In [ ]:
blank_fields = np.array(fields)[mask]

### get data for the blank fields
Things like name, ra, dec, exposure, redshift...

In [ ]:
# get file data
data = load_PSZcatalog()
data = data.sort_values('NAME')

data = redshifts_from_papers(data)

In [ ]:
blank_field_names = [f.split('/')[-1].replace('_', ' ') for f in blank_fields]

In [ ]:
data_mask = [True if f in blank_field_names else False for f in data['NAME']]

In [ ]:
blank_table = Table.from_pandas(data[['NAME', 'RA', 'DEC', 'SNR', 'REDSHIFT']][data_mask])
blank_table.add_column(Column(data=0.0, name='XRT_EXP'))

In [ ]:
outpath = '../data'

for i, name in enumerate(blank_table['NAME']):
    n2 = name.replace(' ', '_')
    
    if os.path.isdir(f'{outpath}/{n2}'):
        evnts = f'{outpath}/{n2}/{n2}_events.fits'
    else:
        continue

    try:
        exp_time = fits.getheader(evnts)['EXPOSURE']
    except FileNotFoundError:
        exp_time = -1

    blank_table['XRT_EXP'][i] = exp_time

In [ ]:
blank_table[(blank_table['REDSHIFT'] > 0) & (blank_table['XRT_EXP'] > 1000)][:20]

In [ ]:
blank_table.filled(-1.0).write('./catalogs/blank_table.fits', overwrite=True)